In [ ]:
# a script for making pretty plots

In [ ]:
# imports
import os
import glob
from functools import reduce
import re

import matplotlib.pyplot as plt
import numpy as np
import csv as csv

from sklearn import linear_model, datasets

%matplotlib inline

In [ ]:
# function for loading one data-set
def load_data(file_path):
    data = np.loadtxt(file_path, delimiter = ',', skiprows=1)
    print(data.shape) 
    if (len(data.shape) < 2):
        data = data[None, :]
    if (data.shape[1] == 0):
        I = np.array([0])
    else:
        I = data[:, -1]
    return I

In [ ]:
# general setup for pretty images 
folder_path = os.getcwd() + '/data/processed/'
image = 'MK4_DPY23-ex-int_mdh1_003'

dataset = []
for path1 in glob.iglob(folder_path + image + "/c" + str(0 + 1) + "-result/*"):
    
    repls = (" - C=0-", " - C=2-"), ("c1-result", "c3-result")
    path2 = reduce(lambda a, kv: a.replace(*kv), repls, path1)
    
    repls = (" - C=0-", " - C=1-"), ("c1-result", "c2-result")
    path3 = reduce(lambda a, kv: a.replace(*kv), repls, path1)

    dataset.append([path1, path2, path3])
    # print(dataset, "processed")


In [ ]:
# check that all files exist
for path in dataset:
    for j in range(0,2):
        if (not os.path.isfile(path[j])):
            print(path, "bad", path[j])

In [ ]:
# actual plotting 
labels = ["dpy23-exon", "dpy23-intron", "mdh-1"]
colors = ["#ffc900", "#56b900", "#0096ff"] # yellow, green, blue
binwidth = [100, 100, 100]

image_name = dataset[0][0][43:67] 

for idx in range(0, len(dataset)):
    fig = plt.figure(figsize=(16,10))
    m = re.search('[^/]+$', dataset[idx][0])
    title = m.group(0)[:14]
    fig.suptitle(image_name + " - " + title, fontsize=14, fontweight='bold')
    for j in range(0, 3): # iterate over all channels
        print(dataset[idx][j])
        I = load_data(dataset[idx][j])
        
        ax = fig.add_subplot(3,1,j + 1)
        ax.set_xlabel('Intensity')
        ax.set_ylabel('# spots')
        ax.hist(I, bins=np.arange(min(I), max(I) + binwidth[j], binwidth[j]), color=colors[j], label=labels[j])
        ax.text(0.9, 0.1, "Total: " + str(I.shape[0]), color='black', bbox=dict(facecolor='white', alpha=1), transform=ax.transAxes)
        ax.set_xlim([-10, 14000])

        if (j == 2):
            ax.set_ylim([0, 2500])
        else:
            ax.set_ylim([0, 200])
        plt.legend(loc = 'upper right')

    if not os.path.exists(folder_path + "histograms/" + image):
        os.makedirs(folder_path + "histograms/" + image)
    plt.savefig(folder_path + "histograms/" + image + "/" + title + ".png")
    plt.show()